In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import text as keras_text, sequence as keras_seq

Using TensorFlow backend.


In [7]:
from keras.models import Sequential,Model,Input,load_model
from keras.layers import LSTM,Dense,TimeDistributed,Activation,MaxPooling2D,Embedding,RepeatVector,Flatten,Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD

In [16]:
data=pd.read_csv('train-bp.csv')  ## read the file

In [17]:
#data.describe()functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions


In [18]:
seq=data["0"]    ## sequence 

In [19]:
y=data.iloc[:,1:]   ###### traget 

In [22]:
###### making 3 word fix sequence

In [23]:
sequence=[]
Length_seq=[]
for ix in range(len(seq)):
    inter_seq=[]
    Seq=seq[ix]
    le=len(Seq)
    k=0
    for ixx in range(int(le/3)):
        f=Seq[k:k+3]
        inter_seq.append(f)
        k=k+3
    Length_seq.append(len(inter_seq))   #### no word present in a sequence 
    sequence.append(inter_seq)

In [24]:
sequence=np.array(sequence)

In [28]:
print(sequence[0][:10])

['MNF', 'LWK', 'GRR', 'FLI', 'AGI', 'LPT', 'FEG', 'AAD', 'EIV', 'DKE']


In [14]:
sns.distplot(Length_seq,axlabel="word in seq length")   #######  

In [45]:
#from gensim.models import Word2Vec

In [1]:
#model = Word2Vec(
 #       sequence,
  #      size=200,
   #     window=3,
    #    min_count=1,
     #   workers=10,sg=1)
#model.train(sequence, total_examples=len(sequence), epochs=10)

In [23]:
#words = list(model.wv.vocab)
#print(words)

In [46]:
#model.wv.save_word2vec_format('skip_gram_3_model.txt',binary=False)
# load model


In [2]:
#embeddings_index = dict()
#f = open('skip_gram_3_model.txt')
#for line in f:
#	values = line.split()
#	word = values[0]
#	coefs = np.asarray(values[1:], dtype='float32')
#	embeddings_index[word] = coefs
#f.close()
#print('Loaded %s word vectors.' % len(embeddings_index))


In [15]:
from keras.preprocessing.text import Tokenizer    ##### tokenize and padding 
tok = Tokenizer(char_level = False)
t=tok.fit_on_texts(sequence)
list_tokenized_train = tok.texts_to_sequences(sequence)
X = keras_seq.pad_sequences(list_tokenized_train, maxlen=350,padding="post") #maxlen padding 100

In [49]:
#embedding_matrix = np.zeros((8001, 100))
#for word, i in tok.word_index.items():
#	embedding_vector = embeddings_index.get(word)
#	if embedding_vector is not None:
#		embedding_matrix[i] = embedding_vector

In [3]:
"""from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

# fit a 2d PCA model to the vectors
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
words=words[:10]
for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()   """

'from gensim.models import Word2Vec\nfrom sklearn.decomposition import PCA\nfrom matplotlib import pyplot\n\n# fit a 2d PCA model to the vectors\nX = model[model.wv.vocab]\npca = PCA(n_components=2)\nresult = pca.fit_transform(X)\n# create a scatter plot of the projection\npyplot.scatter(result[:, 0], result[:, 1])\nwords = list(model.wv.vocab)\nwords=words[:10]\nfor i, word in enumerate(words):\n\tpyplot.annotate(word, xy=(result[i, 0], result[i, 1]))\npyplot.show()   '

In [9]:
from keras.callbacks import EarlyStopping,Callback


In [10]:
def getweights(model):                                  #######  get weight metrix every epochs
    w=model.get_layer(index=1).get_weights()[0]
    print("embedding")
    print(w)   
    w=model.get_layer(index=1).get_weights()[1]
    print("LSTM")
    print(w)
    w=model.get_layer(index=1).get_weights()[2]
    print("dense")
    print(w)

from keras import backend as K
    
def getoutput(model):     ############# get ouutput metrix after every epochs
    
    inp = model.input                                           # input placeholder
    outputs = [layer.output for layer in model.layers]          # all layer outputs
    functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

# Testing
    test = np.random.random(max_len)[np.newaxis,...]
    layer_outs = [func([test, 1.]) for func in functors[0]]
    print("output")
    print(layer_outs)

In [12]:
from keras.callbacks import LambdaCallback
pw = LambdaCallback(on_epoch_end=lambda batch, logs:getweights(model))
out = LambdaCallback(on_epoch_end=lambda batch, logs:getoutput(model))

In [13]:
max_len=350
output=932

In [14]:
model= Sequential()
model.add(Embedding(8001,100,input_length=max_len,trainable=True))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(932,activation="sigmoid"))
model.compile( optimizer = 'rmsprop', loss = "binary_crossentropy",metrics = ['accuracy'] )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 350, 100)          800100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 932)               60580     
Total params: 902,920
Trainable params: 902,920
Non-trainable params: 0
_________________________________________________________________


In [17]:
history=model.fit(X,y, batch_size =128,epochs =300,class_weight='balanced',validation_split=0.35,shuffle=True,callbacks=[pw])

Train on 23647 samples, validate on 12733 samples
Epoch 1/500
23647/23647 [==============================] - 100s 4ms/step - loss: 0.1577 - acc: 0.9627 - val_loss: 0.1167 - val_acc: 0.9681
embedding
[[ 0.04840998  0.08138335 -0.10710852 ...  0.02849966  0.01640376
   0.12292346]
 [ 0.04367676 -0.00827236  0.07333081 ...  0.05091657 -0.07733617
  -0.05345358]
 [-0.0946527   0.00724279  0.0078496  ...  0.10160974 -0.06043201
   0.15707763]
 ...
 [ 0.12602091  0.06489238 -0.01201935 ... -0.02095357  0.07773806
  -0.01221271]
 [-0.00400775 -0.10664369  0.08022289 ...  0.06115166  0.13697171
   0.05471778]
 [ 0.10388743  0.04610727  0.05297854 ... -0.09113459 -0.13025698
  -0.04234228]]
LSTM
[[-0.00820597  0.00285968 -0.08349665 ... -0.02678023 -0.06696301
  -0.0540817 ]
 [-0.06578734  0.06376179  0.10378426 ... -0.06978371  0.15003803
  -0.0843235 ]
 [-0.00909365  0.00620483 -0.08024006 ... -0.12046806 -0.02191466
  -0.02615907]
 ...
 [ 0.0304428  -0.01839438  0.03336089 ... -0.02378081 -0

23647/23647 [==============================] - 96s 4ms/step - loss: 0.1182 - acc: 0.9680 - val_loss: 0.1171 - val_acc: 0.9680
embedding
[[ 0.05033695  0.08138704 -0.10710792 ...  0.03134738  0.01892247
   0.12292366]
 [ 0.04208357 -0.00827534  0.07333035 ...  0.04816036 -0.07970158
  -0.05345373]
 [-0.0938902   0.00724372  0.00784983 ...  0.10395314 -0.05841402
   0.15707761]
 ...
 [ 0.12673645  0.06489323 -0.01201907 ... -0.01870921  0.07962916
  -0.0122127 ]
 [-0.00280958 -0.10664211  0.08022314 ...  0.06371648  0.13906123
   0.05471788]
 [ 0.10280357  0.04610546  0.05297828 ... -0.09370159 -0.13242123
  -0.04234229]]
LSTM
[[-0.01306999  0.00281563 -0.08350421 ... -0.02987748 -0.07003769
  -0.05406982]
 [-0.06091809  0.06380583  0.10379183 ... -0.06668543  0.1531128
  -0.08433537]
 [-0.01396266  0.0061608  -0.0802476  ... -0.12356576 -0.02498958
  -0.02614731]
 ...
 [ 0.02558205 -0.01843844  0.03335334 ... -0.02687857 -0.10201462
  -0.14044346]
 [-0.07123435 -0.02840776 -0.03620533 .

23647/23647 [==============================] - 95s 4ms/step - loss: 0.1181 - acc: 0.9680 - val_loss: 0.1171 - val_acc: 0.9681
embedding
[[ 0.05051517  0.08142921 -0.10701475 ...  0.03528807  0.02187527
   0.12770812]
 [ 0.04191471 -0.00831432  0.0732477  ...  0.04420138 -0.08262467
  -0.05828118]
 [-0.09386615  0.00724985  0.00786751 ...  0.10934583 -0.05610662
   0.1614834 ]
 ...
 [ 0.12675092  0.06489427 -0.01200912 ... -0.01737406  0.08161408
  -0.01095195]
 [-0.00274108 -0.1066303   0.08026499 ...  0.06855068  0.14180532
   0.05942113]
 [ 0.10271118  0.04608449  0.05292801 ... -0.09830578 -0.13521744
  -0.04704997]]
LSTM
[[-0.01459922  0.00229195 -0.08445688 ... -0.03279131 -0.07317289
  -0.05887628]
 [-0.05938752  0.06432979  0.10474552 ... -0.06377248  0.15624793
  -0.07952967]
 [-0.01549514  0.00563642 -0.08120268 ... -0.12647778 -0.02812486
  -0.03095213]
 ...
 [ 0.02411862 -0.01895199  0.03243699 ... -0.02991517 -0.10514889
  -0.14526084]
 [-0.0718217  -0.02860046 -0.03670447 

23647/23647 [==============================] - 95s 4ms/step - loss: 0.1181 - acc: 0.9680 - val_loss: 0.1171 - val_acc: 0.9681
embedding
[[ 0.05143133  0.08144213 -0.10700724 ...  0.03808741  0.02400405
   0.12770809]
 [ 0.04089921 -0.00832906  0.07323876 ...  0.04136503 -0.08481283
  -0.0582808 ]
 [-0.0939486   0.00724914  0.00786691 ...  0.10822571 -0.05629245
   0.16148373]
 ...
 [ 0.1268355   0.06489655 -0.01200754 ... -0.01592069  0.08151629
  -0.01095181]
 [-0.00300346 -0.10663172  0.08026404 ...  0.06652787  0.14077243
   0.05942132]
 [ 0.10242562  0.04608008  0.05292571 ... -0.10057315 -0.13631712
  -0.04704986]]
LSTM
[[-0.01720625  0.00212096 -0.08456008 ... -0.03590725 -0.07619609
  -0.05882412]
 [-0.05677425  0.06450111  0.10484888 ... -0.06065661  0.15927126
  -0.07958174]
 [-0.01810798  0.00546516 -0.081306   ... -0.12959349 -0.03114821
  -0.03089957]
 ...
 [ 0.02150737 -0.01912325  0.0323336  ... -0.03303079 -0.10817236
  -0.14520851]
 [-0.07278574 -0.02862553 -0.03671727 

23647/23647 [==============================] - 95s 4ms/step - loss: 0.1180 - acc: 0.9680 - val_loss: 0.1167 - val_acc: 0.9681
embedding
[[ 0.02907727  0.08144717 -0.10699629 ...  0.03808752  0.02400409
   0.12770829]
 [ 0.06257597 -0.00833508  0.07322684 ...  0.04136507 -0.08481283
  -0.05828064]
 [-0.10099618  0.00724809  0.00786555 ...  0.10822563 -0.05629249
   0.16148353]
 ...
 [ 0.12903064  0.06489692 -0.01200735 ... -0.0159207   0.0815163
  -0.01095176]
 [ 0.01065602 -0.10663333  0.08026058 ...  0.06652791  0.14077245
   0.05942148]
 [ 0.11864231  0.04607867  0.05292309 ... -0.10057309 -0.13631709
  -0.04704969]]
LSTM
[[ 0.01446613  0.00205227 -0.08467451 ... -0.03589716 -0.07619286
  -0.05878701]
 [-0.08739205  0.06456663  0.10497443 ... -0.06067174  0.15926623
  -0.07963013]
 [ 0.01242802  0.00539913 -0.08143298 ... -0.12957835 -0.03114318
  -0.03085082]
 ...
 [ 0.05204357 -0.01918921  0.03220663 ... -0.03301525 -0.10816722
  -0.14515896]
 [-0.04291473 -0.02862295 -0.03673277 .

...

23647/23647 [==============================] - 95s 4ms/step - loss: 0.0750 - acc: 0.9758 - val_loss: 0.1815 - val_acc: 0.9629
embedding
[[ 0.09276588  0.02652019 -0.1077271  ...  0.10929123 -0.05529933
   0.04466749]
 [-0.00218777 -0.02257522 -0.05233954 ... -0.13208134 -0.17219935
  -0.09309503]
 [-0.13762273  0.00935086  0.8054566  ...  0.36653757 -0.40250903
   0.50913   ]
 ...
 [ 0.14743924  0.12337983 -0.02773886 ...  0.03453455 -0.18061303
   0.17199911]
 [ 0.13629322 -0.05910794  0.08277185 ... -0.09363695 -0.06401745
   0.31351095]
 [ 0.3409131   0.25754014  0.47435766 ... -0.20374715 -0.26145908
   0.06837802]]
LSTM
[[ 0.37904143  0.28705707  0.09117675 ...  0.2980293  -0.07576869
   0.03420135]
 [-0.40199372  0.08066639  0.1180796  ...  0.04643873  0.35620427
  -0.04782894]
 [ 0.1796962   0.290186   -0.06969173 ... -0.294991   -0.7252104
   0.16285388]
 ...
 [ 0.1646248   0.61340433  0.12976834 ... -0.09404992 -0.6782193
  -0.01342132]
 [ 0.12276496  0.22848323  0.03390871 ..

23647/23647 [==============================] - 95s 4ms/step - loss: 0.0748 - acc: 0.9760 - val_loss: 0.1805 - val_acc: 0.9632
embedding
[[ 0.09089273  0.03146178 -0.11114484 ...  0.10876593 -0.0542989
   0.04415341]
 [ 0.00116106 -0.03207707 -0.04353707 ... -0.12471695 -0.16921124
  -0.08702381]
 [-0.13349462  0.00848253  0.7915267  ...  0.3589953  -0.41738805
   0.50331765]
 ...
 [ 0.1379431   0.1223071  -0.03340196 ...  0.0296704  -0.18958943
   0.1662748 ]
 [ 0.13731107 -0.05250258  0.07753681 ... -0.09706939 -0.05819845
   0.3175498 ]
 [ 0.3524465   0.26230055  0.492338   ... -0.22539939 -0.25006324
   0.06859499]]
LSTM
[[ 0.38762584  0.28917676  0.09107883 ...  0.30114952 -0.06056155
   0.03231101]
 [-0.41304415  0.07945219  0.09490667 ...  0.0575988   0.33491114
  -0.04589986]
 [ 0.18160687  0.30318347 -0.05971701 ... -0.27495214 -0.70928395
   0.16218053]
 ...
 [ 0.16327982  0.6336068   0.13883862 ... -0.0979564  -0.68257815
  -0.00575087]
 [ 0.12938648  0.22989884  0.03998962 .

23647/23647 [==============================] - 95s 4ms/step - loss: 0.0746 - acc: 0.9759 - val_loss: 0.1831 - val_acc: 0.9630
embedding
[[ 0.10181046  0.03612696 -0.10358185 ...  0.10652041 -0.0570376
   0.05299173]
 [ 0.00136438 -0.03541702 -0.02250982 ... -0.13968578 -0.20418812
  -0.08149897]
 [-0.12984726  0.00619857  0.7969825  ...  0.35945368 -0.409243
   0.5117813 ]
 ...
 [ 0.14301433  0.11874431 -0.03376408 ...  0.03735186 -0.17115465
   0.16980456]
 [ 0.14048587 -0.05439378  0.07482234 ... -0.09465878 -0.04420437
   0.319209  ]
 [ 0.35682797  0.2590251   0.48446444 ... -0.21764754 -0.24460845
   0.05592877]]
LSTM
[[ 0.38578868  0.28532177  0.09348831 ...  0.2954696  -0.06678189
   0.03311097]
 [-0.40252462  0.08257353  0.09013502 ...  0.07691044  0.3356697
  -0.03746165]
 [ 0.16505161  0.30653563 -0.07125871 ... -0.289838   -0.7200589
   0.15874633]
 ...
 [ 0.17207253  0.64512473  0.1468801  ... -0.09992169 -0.71087235
  -0.01166061]
 [ 0.11712479  0.21463464  0.02860794 ...  

23552/23647 [============================>.] - ETA: 0s - loss: 0.0744 - acc: 0.9760

KeyboardInterrupt: 

In [4]:
#inp = model.input                                           # input placeholder
#outputs = [layer.output for layer in model.layers]          # all layer outputs
#functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    # evaluation functions

# Testing
#test = np.random.random(max_len)[np.newaxis,...]
#layer_outs = [func([X_test, 2.]) for func in functors[:1]]
#print("output")
#print(layer_outs)

In [22]:
model.save("final_char.hdf5")

In [19]:
#from keras.models import Model           ##### 2nd method to get output metrix by layer name 
#layer_name='dense_1'
#im = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
#io=im.predict(X)